In [1]:
from transformers import TapasConfig, TapasForQuestionAnswering

# for example, the base sized model with default SQA configuration
# model = TapasForQuestionAnswering.from_pretrained("google/tapas-base")

# or, the base sized model with WTQ configuration
config = TapasConfig.from_pretrained("google/tapas-base-finetuned-wtq")
#tapas-large-finetuned-wtq
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base", config=config)

# or, the base sized model with WikiSQL configuration
#config = TapasConfig("google-base-finetuned-wikisql-supervised")
#model = TapasForQuestionAnswering.from_pretrained("google/tapas-base", config=config)

/opt/conda/envs/research/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 1.66k/1.66k [00:00<00:00, 3.36MB/s]
pytorch_model.bin: 100%|██████████| 443M/443M [00:18<00:00, 23.5MB/s] 
Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['aggregation_classifier.bias', 'aggregation_classifier.weight', 'column_output_bias', 'column_output_weights', 'output_bias', 'output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
from transformers import TapasTokenizer
import pandas as pd

model_name = "google/tapas-base"
tokenizer = TapasTokenizer.from_pretrained(model_name)

data = {"Actors": ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], "Number of movies": ["87", "53", "69"]}
queries = [
    "What is the name of the first actor?",
    "How many movies has George Clooney played in?",
    "What is the total number of movies?",
]
answer_coordinates = [[(0, 0)], [(2, 1)], [(0, 1), (1, 1), (2, 1)]]
answer_text = [["Brad Pitt"], ["69"], ["209"]]
table = pd.DataFrame.from_dict(data)
inputs = tokenizer(
    table=table,
    queries=queries,
    answer_coordinates=answer_coordinates,
    answer_text=answer_text,
    padding="max_length",
    return_tensors="pt",
)
inputs

tokenizer_config.json: 100%|██████████| 490/490 [00:00<00:00, 1.08MB/s]
vocab.txt: 100%|██████████| 262k/262k [00:00<00:00, 852kB/s]
special_tokens_map.json: 100%|██████████| 154/154 [00:00<00:00, 355kB/s]
config.json: 100%|██████████| 1.52k/1.52k [00:00<00:00, 3.54MB/s]
/opt/conda/envs/research/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2762: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/conda/envs/research/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1561: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]

{'input_ids': tensor([[ 101, 2054, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2116,  ...,    0,    0,    0],
        [ 101, 2054, 2003,  ...,    0,    0,    0]]), 'labels': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'numeric_values': tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]]), 'numeric_values_scale': tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]]), 'token_type_ids': tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

In [12]:
import torch

outputs = model(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            token_type_ids=inputs.token_type_ids,
            labels=inputs.labels,
            numeric_values=inputs.numeric_values,
            numeric_values_scale=inputs.numeric_values_scale,
            float_answer=torch.tensor([[torch.nan], [69], [209]]),
        )

outputs

TableQuestionAnsweringOutput(loss=tensor(0.8520, grad_fn=<AddBackward0>), logits=tensor([[-10000., -10000., -10000.,  ..., -10000., -10000., -10000.],
        [-10000., -10000., -10000.,  ..., -10000., -10000., -10000.],
        [-10000., -10000., -10000.,  ..., -10000., -10000., -10000.]],
       grad_fn=<ViewBackward0>), logits_aggregation=tensor([[ 0.1940,  0.3279,  0.4199, -0.1130],
        [ 0.1130, -0.3987, -0.3263, -0.3344],
        [ 0.5499, -0.2853, -0.5349,  0.5817]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [13]:
from transformers import TapasTokenizer, TapasForQuestionAnswering
import pandas as pd

model_name = "google/tapas-base-finetuned-wtq"
model = TapasForQuestionAnswering.from_pretrained(model_name)
tokenizer = TapasTokenizer.from_pretrained(model_name)

data = {"Actors": ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], "Number of movies": ["87", "53", "69"]}
queries = [
    "What is the name of the first actor?",
    "How many movies has George Clooney played in?",
    "What is the total number of movies?",
]
table = pd.DataFrame.from_dict(data)
inputs = tokenizer(table=table, queries=queries, padding="max_length", return_tensors="pt")
outputs = model(**inputs)
predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions(
    inputs, outputs.logits.detach(), outputs.logits_aggregation.detach()
)

# let's print out the results:
id2aggregation = {0: "NONE", 1: "SUM", 2: "AVERAGE", 3: "COUNT"}
aggregation_predictions_string = [id2aggregation[x] for x in predicted_aggregation_indices]

answers = []
for coordinates in predicted_answer_coordinates:
    if len(coordinates) == 1:
        # only a single cell:
        answers.append(table.iat[coordinates[0]])
    else:
        # multiple cells
        cell_values = []
        for coordinate in coordinates:
            cell_values.append(table.iat[coordinate])
        answers.append(", ".join(cell_values))

display(table)
print("")
for query, answer, predicted_agg in zip(queries, answers, aggregation_predictions_string):
    print(query)
    if predicted_agg == "NONE":
        print("Predicted answer: " + answer)
    else:
        print("Predicted answer: " + predicted_agg + " > " + answer)

pytorch_model.bin: 100%|██████████| 443M/443M [00:04<00:00, 109MB/s]  
tokenizer_config.json: 100%|██████████| 490/490 [00:00<00:00, 1.17MB/s]
vocab.txt: 100%|██████████| 262k/262k [00:00<00:00, 881kB/s]
special_tokens_map.json: 100%|██████████| 154/154 [00:00<00:00, 397kB/s]
/opt/conda/envs/research/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2762: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/conda/envs/research/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1561: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc

,Actors,Number of movies
0,Brad Pitt,87
1,Leonardo Di Caprio,53
2,George Clooney,69



What is the name of the first actor?
Predicted answer: Brad Pitt
How many movies has George Clooney played in?
Predicted answer: COUNT > 69
What is the total number of movies?
Predicted answer: SUM > 87, 53, 69


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd

data = {"Actors": ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], "Number of movies": ["87", "53", "69"]}
queries = [
    "What is the name of the first actor?",
    "How many movies has George Clooney played in?",
    "What is the total number of movies?",
]
table = pd.DataFrame.from_dict(data)

tokenizer = AutoTokenizer.from_pretrained("seeklhy/codes-1b")
model = AutoModelForCausalLM.from_pretrained("seeklhy/codes-1b")

/tmp/ipykernel_4097985/126750372.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
tokenizer_config.json: 100%|██████████| 717/717 [00:00<00:00, 2.19MB/s]
vocab.json: 100%|██████████| 777k/777k [00:00<00:00, 1.94MB/s]
merges.txt: 100%|██████████| 442k/442k [00:00<00:00, 27.7MB/s]
tokenizer.json: 100%|██████████| 2.06M/2.06M [00:00<00:00, 17.5MB/s]
special_tokens_map.json: 100%|██████████| 564/564 [00:00<00:00, 1.28MB/s]
config.json: 100%|██████████| 1.02k/1.02k [00:00<00:00, 2.31MB/s]
pytorch_model.bin: 100%|██████████| 4.55G/4.55G [03:43

In [42]:
in_text = """
table movie , columns = [ movie.mid ( int | primary key | comment : movie id | values : 101 ,
102 ) , movie.title ( text | values : Gone with the Wind , Star Wars ) , movie.year ( int |
values : 1939 , 1977 ) , movie.director ( text | values : Victor Fleming , George Lucas ) ]
table reviewer , columns = [ reviewer.rid ( int | primary key | comment : reviewer id | values :
201 , 202 ) , reviewer.name ( text | values : Sarah Martinez , Daniel Lewis ) ]
table rating , columns = [ rating.rid ( int | comment : reviewer id | values : 201 , 202 ) ,
rating.mid ( int | comment : movie id | values : 101 ,106 ) , rating.stars ( int | comment : rating
stars | values : 2 , 4 ) , rating.ratingdate ( date | values : 2011-01-22 , 2011-01-27 ) ]
foreign keys :
rating.rid = reviewer.rid
rating.mid = movie.mid
matched values :
reviewer.name ( Sarah Martinez )
Question:
What are the names of all directors whose movies have been reviewed by Sarah Martinez?
"""

#queries.append("What is the average of column number of movies given that the column actor has value Pumuckel?")

in_text = f"""table movie , columns = [ movie.Actor ( text | primary key | comment : name of | values : Olaf Scholz ,
Sinatra ) , movie.Number_of_Movies ( int | values : 9 , 12 ) ] Question: {queries[-1]}"""

in_text = queries[-1]

inputs = tokenizer(in_text, return_tensors="pt")

In [43]:
outputs = model(**inputs)
tokenizer.batch_decode(model.generate(**inputs, max_new_tokens=64, do_sample=False))[0]

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


"What is the average of column number of movies given that the column actor has value Pumuckel?\nSELECT AVG(column_number) FROM movies WHERE column_actor = 'Pumuckel';<|endoftext|>"

In [30]:
tokenizer.decode(outputs['logits'].argmax(dim=-1)[0])

'(__ table id titlemovie_id,int ) string key ) auto\'\' id ) auto : 1,0, 102,, movie.name ( string | primary : "host with the Wind, The Wars ), movie.year ( int | valuesvalues : 2977, 2947 ), movie.rating ( text | values : <irgin Hleming, <orge Lucas ),\n\n actor, columns = [ reviewer.rid ( int | primary key | comment : reviewer id |\n :\n101, 202 ), reviewer.name ( text | values : <ah,inez, <leewis ),\ntable rating, columns = [ rating.rid ( int | primary : reviewer id | values : 201, 202 ), ratingrating.mid ( int | comment : movie id | values : 101, 02 ), rating.rating ( int | values : stars || | values : 5, 4 ) ] rating.date ( ( date | comment : 2012-01-00, 2011-01-23 ) ]\ntable key (\nmovie.rid ( reviewer.rid\nrating.mid = movie.mid\n\n by :\nrating.rid = Sarah Martinez,,movie  WhatWhat is the top of the theors who movies have been r by moreah Martinez?\nSELECT'